In [ ]:
import pandas as pd # type: ignore
import firebase_admin # type: ignore
from firebase_admin import credentials # type: ignore
from firebase_admin import firestore # type: ignore

cred = credentials.Certificate('../../Documents/GoogleCerts/astroworld-trevor-firebase-adminsdk-l05ps-820638c3ff.json')
firebase_admin.initialize_app(cred)
db = firestore.client()

table = pd.read_csv('./hygdata_v41.csv')
table = table.dropna(subset=['proper'])
table = table[table['proper'] != 'Sol']
stars_table = table[['id', 'proper', 'ra', 'dec', 'mag', 'spect']].rename(columns={
    'id': 'id',
    'proper': 'name',
    'ra': 'rightAscension',
    'dec': 'declination',
    'mag': 'magnitude',
    'spect': 'spectralType'
})

stars_dict = stars_table.to_dict(orient='records')
stars_ref = db.collection('stars')
for star in stars_dict:
    star_ref = stars_ref.document(str(int(star['id'])))
    star_ref.set(star)

print("Data uploaded successfully!")

In [1]:
import warnings # type: ignore
warnings.filterwarnings('ignore') # type: ignore
from datetime import date, timedelta  # type: ignore
import firebase_admin # type: ignore
from firebase_admin import credentials, firestore # type: ignore
import os  # type: ignore
from skyfield.api import load  # type: ignore
from skyfield.magnitudelib import planetary_magnitude  # type: ignore

cred = credentials.Certificate('../../Documents/GoogleCerts/astroworld-trevor-firebase-adminsdk-l05ps-820638c3ff.json')
firebase_admin.initialize_app(cred)
db = firestore.client()

try:
    os.remove('de421.bsp')
except FileNotFoundError:
    pass

ts = load.timescale()
planets = load('de421.bsp')
t = ts.utc(date.today() + timedelta(days=1))

solar_system_objects = {
    'Sun': planets['sun'],
    'Moon': planets['moon'],
    'Mercury': planets['mercury BARYCENTER'],
    'Venus': planets['venus BARYCENTER'],
    'Mars': planets['mars BARYCENTER'],
    'Jupiter': planets['jupiter BARYCENTER'],
    'Saturn': planets['saturn BARYCENTER'],
    'Uranus': planets['uranus BARYCENTER'],
    'Neptune': planets['neptune BARYCENTER']
}

def get_planet_data(planet, name, id, spect):
    earth = planets['earth']
    astrometric = earth.at(t).observe(planet)
    ra, dec, _ = astrometric.radec()
    if name == 'Sun':
        mag = -26.74
    elif name == 'Moon':
        mag = -12.74
    else:
        mag = planetary_magnitude(earth.at(t).observe(planet))
    return {
        'id': id,
        'name': name,
        'rightAscension': ra.hours,
        'declination': dec.degrees,
        'magnitude': mag,
        'spectralType': chr(spect)
    }

planet_data = []
id = 0.0
spect = ord('r')
for name, planet in solar_system_objects.items():
    data = get_planet_data(planet, name, id, spect)
    planet_data.append(data)
    id += 1.0
    spect += 1

stars_ref = db.collection('stars')
for name in solar_system_objects.keys():
    query = stars_ref.where('name', '==', name)
    docs = query.get()
    for doc in docs:
        doc.reference.delete()

for data in planet_data:
    data['rightAscension'] = float(data['rightAscension'])
    data['declination'] = float(data['declination'])
    data['magnitude'] = float(data['magnitude'])
    
    stars_ref.document(str(int(data['id']))).set(data)

print("Solar system objects updated in Firestore!")

[#################################] 100% de421.bsp


Solar system objects updated in Firestore!
